In [205]:
from gensim.models.word2vec import Word2Vec
import word2vecReader
from scipy.stats import mode
from numpy import mean
from numpy import median
from numpy import std
from textblob import TextBlob
from nltk.corpus import stopwords
from sklearn.utils import shuffle
import pandas as pd
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from tqdm import tqdm
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from textblob.taggers import PatternTagger
import re
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from ParserSarcasmDetector import ParserSarcasmDetector

In [206]:
sarc = pd.read_csv("data/SarcasmAddData.csv")

In [203]:
import sys
stdin, stdout = sys.stdin, sys.stdout
reload(sys)
sys.stdin, sys.stdout = stdin, stdout
sys.setdefaultencoding('utf-8')

In [3]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [4]:
model2 = word2vecReader.Word2Vec.load_word2vec_format('word2vec_twitter_model/word2vec_twitter_model.bin', binary=True)

In [5]:
affect = pd.read_csv("sp_files/Ratings_Warriner_et_al-2.csv")
keys = list(affect["Word"])
values = list(affect["V.Mean.Sum"])
affect_scores = dict(zip(keys, values))

In [6]:
def get_set(f):
    s = set()
    set_file = open(f, "r")
    for line in set_file:
        l = line.split()
        s.add(l[0])
    return s

swear_words_set = get_set("sp_files/swear_words_list.txt")

In [7]:
def get_strength_dict(f, val):
    s_d = {}
    s_file = open(f, "r")
    for line in s_file:
        l = line.split()
        if val:
            s_d[re.sub(r"\*",r"",l[0])] = int(l[1])
        else:
            s_d[re.sub(r"\*",r"",l[0])] = 0
    return s_d

sentiments_dict = get_strength_dict("sp_files/SentimentLookupTable.txt", True)

In [8]:
def get_strength_list(f):
    s_l = []
    s_file = open(f, "r")
    for line in s_file:
        s_l.append(line.strip())
    return s_l

In [9]:
booster_dict = get_strength_dict("sp_files/BoosterWordList.txt", True)
idioms_dict = get_strength_list("sp_files/EC-Idioms-Intermediate-Advanced-3.txt")
slang_dict = get_strength_dict("sp_files/SlangLookupTable.txt", False)
emoticon_dict = get_strength_list("sp_files/EmoticonLookupTable.txt")

In [115]:
def get_data_set(filename):
    data = pd.read_csv(filename, encoding="utf-8", engine='python')
    data = data[["text", "sarc"]]
    data = data[data["text"].notnull()]
    data = data[data["sarc"].notnull()]
    data["sarc"] = data["sarc"].apply(lambda x: int(x))
    data = data.drop_duplicates()
    sarc = data[data["sarc"]==1]
    not_sarc = data[data["sarc"]==0][:len(data[data["sarc"]==1])]
    data_1_1 = pd.concat([sarc, not_sarc], ignore_index=True)
    data_1_1 = shuffle(data_1_1)
    return data_1_1

In [116]:
data = get_data_set("sarcasm_set_small.csv")

In [117]:
def senti_score(x):
    wl = WordNetLemmatizer()
    ps = PorterStemmer()
    if x in sentiments_dict:
        return sentiments_dict[x]
    lemma  = wl.lemmatize(x)
    if lemma in sentiments_dict:
        return sentiments_dict[lemma]
    stem = ps.stem(x)
    if stem in sentiments_dict:
        return sentiments_dict[stem]
    return 0.0

In [118]:
def check_swear_words(t):
    for w in t:
        if w[0] in swear_words_set:
            return 1
    return 0

In [119]:
def get_distribution_f(df, col):
    df = df.copy()
    df[col+"mean"]= df[col].apply(lambda x: mean(x or [0]))
    df[col+"median"]= df[col].apply(lambda x: median(x or [0]))
    df[col+"mode"]= df[col].apply(lambda x: mode(x or [0], axis=None).mode[0])
    df[col+"std"]= df[col].apply(lambda x: std(x or [0]))
    df[col+"max"]= df[col].apply(lambda x: max(x or [0]))
    df[col+"min"]= df[col].apply(lambda x: min(x or [0]))
    df = df.drop(col, axis=1)
    return df

In [120]:
def get_mean_vector(text, vec_len, m):
    vec = [0] * vec_len
    count = 0
    words = TextBlob(text.lower()).words
    for w in words:
        if w in m:
            vec += m[w]
            count += 1
    if count == 0:
        return vec
    return vec / (1.0 * count)

In [121]:
def get_sim_d(words):
    d = []
    stop = stopwords.words('english')
    words = [i for i in words if i not in stop]
    #print words
    w = 5
    for i in range(len(words)-1):
        if i + 1 + w > len(words):
            r = len(words)
        else:
            r = i + 1 + w
        for j in range(i+1, r):
            if (words[i] in model) and (words[j] in model):
                #print (words[i], words[j]), model.similarity(words[i], words[j])
                d.append(model.similarity(words[i], words[j]))
    return d        

In [122]:
def get_mean_vector_f(df, vec_len, m, suf):
    df = df.copy()
    df["v"] = df["text"].apply(lambda x: get_mean_vector(x, vec_len, m))
    for i in range(vec_len):
        df["mean_v_"+str(i+1)+suf]=df["v"].apply(lambda x: x[i])
    df = df.drop("v", axis=1)
    return df

In [123]:
def get_similarity_f(df):
    df = df.copy()
    df["sim_d"] = df["words"].apply(lambda x: get_sim_d(x))
    df = get_distribution_f(df, "sim_d")
    return df

In [124]:
def get_similarity(words, word):
    new_words = [i for i in words if i in model]
    if len(new_words) == 0:
        return 0
    return model.n_similarity(new_words, [word])

In [125]:
def get_match(x):
    new_words = [i for i in x if i in model]
    if len(new_words) == 0:
        return float(senti_score(" "))
    return float(senti_score(model.doesnt_match(new_words)))

def get_word2vec_features(df):
    df = df.copy()
    df["senti_not_match"] = df["words"].apply(lambda x: get_match(x))
    df['sim_sarc'] = df["words"].apply(lambda x: get_similarity(x, "sarcasm"))
    df['sim_happy'] = df["words"].apply(lambda x: get_similarity(x, "happy"))
    df['sim_sad'] = df["words"].apply(lambda x: get_similarity(x, "sad"))
    df['sim_angry'] = df["words"].apply(lambda x: get_similarity(x, "angry"))
    return df

In [126]:
def iden_pronouns(t1, t2):
    common_words = list(set(t1).intersection(set(t2)))
    for i in common_words:
        if i[1] != "PRP" or i[1] != "PRP$":
            return True
    return False

def iden_str(t1, t2):
    stop = stopwords.words()
    t1 = [i[0] for i in t1 if i[1] not in stop]
    t2 = [i[0] for i in t2 if i[1] not in stop]
    common_words = list(set(t1).intersection(set(t2)))
    if len(common_words) != 0:
        return True
    return False


def check_coherence(text):
    blob = TextBlob(text)
    sentences = blob.sentences
    if len(sentences) <= 1:
        return 0
    demonstrative = ["this", "that", "these", "those"]
    tagged_set = [TextBlob(str(sentence).lower()).tags for sentence in sentences]
    tagged_set = [i for i in tagged_set if len(i) != 0]
    for i in range(len(tagged_set) - 1):
        if iden_pronouns(tagged_set[i], tagged_set[i+1]) or iden_str(tagged_set[i], tagged_set[i+1])\
           or  (tagged_set[i+1][0][0] == "the") or (tagged_set[i+1][0][0] in demonstrative):
                continue
        else:
            return -1
    return 1

In [127]:
def get_score(word):
    scores = []
    ss = senti_score(word)
    sn = TextBlob(word + " ").polarity * 5.0
    if ss != 0:
        scores.append(ss)
    if sn != 0:
        scores.append(sn)
    return np.mean(scores or [0])
        
        
def get_scores(filtered_text):
    scores = [get_score(w[0]) for w in filtered_text]
    pos_scores = [s for s in scores if s > 0]
    neg_scores = [n for n in scores if n < 0]
    return (sum(pos_scores or [0]), sum(neg_scores or [0]))
    

def get_emotional_scores(df):
    df = df.copy()
    df["sn"] = df["tagged_words"].apply(lambda x: get_scores(x))
    df["sum_pos_score"] = df["sn"].apply(lambda x: x[0])
    df["sum_neg_score"] = df["sn"].apply(lambda x: x[1])
    df = df.drop("sn", axis=1)
    return df

In [128]:
def get_sentiment_features(df):
    df = df.copy()
    df["pos_low"] = df["sum_pos_score"].apply(lambda x: int(x<= 0))
    df["pos_medium"] = df["sum_pos_score"].apply(lambda x: int((x > 0) and (x <= 1)))
    df["pos_high"] = df["sum_pos_score"].apply(lambda x: int(x >= 2))
    df["neg_low"] = df["sum_pos_score"].apply(lambda x: int(abs(x) <= 0))
    df["neg_medium"] = df["sum_pos_score"].apply(lambda x:  int((abs(x) > 0) and (abs(x) <= 1)))
    df["neg_high"] = df["sum_pos_score"].apply(lambda x: int(abs(x) >= 2))
    df = df.drop("sum_pos_score", axis=1)
    df = df.drop("sum_neg_score", axis=1)
    return df

In [130]:
def get_rep_punct(text):
    punct = [',', '.', '?', '!']
    count = 0
    for i in range(len(text)-1):
        if (text[i] in punct) and (text[i] == text[i+1]):
            count+=1
    return count


def get_rep_chars(text):
    count = 0
    for i in range(len(text)-1):
        if (text[i].isalpha()) and (text[i] == text[i+1]):
            count+=1
    return count

In [131]:
def get_punct_3_f(df, col_list):
    df = df.copy()
    for col in col_list:
        df[col + "_low"] = df[col].apply(lambda x: int(x ==0))
        df[col + "_medium"] = df[col].apply(lambda x: int((x >= 1) and (x <= 3)))
        df[col + "_high"] = df[col].apply(lambda x: int(x >= 4))
        df = df.drop(col, axis=1)
    return df

In [132]:
def get_punctuation_and_sp_simbols_feature(df):
    df = df.copy()
    df["idioms_number"] = df["text"].apply(lambda x: len([i for i in idioms_dict if i in x]))
    df["exlamation_number"] = df["text"].apply(lambda x: x.count("!"))
    df["slang_and_booster"] = df["text"].apply(lambda x: len([i for i in slang_dict if i in x.lower()]) + 
                                                             len([i for i in booster_dict if i in x.lower()]))
    df["original_tagged_words"] = df["text"].apply(lambda x: TextBlob(x.decode("utf-8"),pos_tagger=PatternTagger()).tags)
    df["number_capitalized"] = df["original_tagged_words"].apply(lambda x: len([i for i in x if i[0].isupper()]))
    df = df.drop("original_tagged_words", axis=1)
    df["emoticons_number"] = df["text"].apply(lambda x: len([i for i in emoticon_dict if i in x]))
    df["rep_punc"] = df["text"].apply(lambda x: get_rep_punct(x))
    df["rep_chars"] = df["text"].apply(lambda x: get_rep_chars(x))
    df = get_punct_3_f(df, ["idioms_number", "exlamation_number", "slang_and_booster", "number_capitalized",
                            "emoticons_number", "rep_punc", "rep_chars"])
    return df

In [133]:
def get_sentiment(x):
    s = [senti_score(i[0]) for i in x]
    return max(s or [0]) - min(s or [0])

def get_affect_score(x):
    s = [affect_scores[i[0]] for i in x if i[0] in affect_scores]
    return max(s or [0]) - min(s or [0])

def get_contrasting_features(df):
    df = df.copy()
    df["del_sentiment"] = df["tagged_words"].apply(lambda x: get_sentiment(x))
    df["del_affect"] = df["tagged_words"].apply(lambda x: get_affect_score(x))
    return df

In [134]:
def check_swear_words(t):
    for w in t:
        if w[0] in swear_words_set:
            return 1
    return 0

In [215]:
def get_iws(x, clf):
    a = clf.iws(x)
    if a == 0:
        a = -1
    elif a is None:
        a = 0
    return a


def create_features(df):
    df = df.copy()
    df["words"] = df["text"].apply(lambda x: TextBlob(x.lower()).words)
    df["tagged_words"] = df["text"].apply(lambda x: TextBlob(x.encode("utf-8").lower(),pos_tagger=PatternTagger()).tags)
    df["length_d"] = df["words"].apply(lambda x: [len(i) for i in x])
    df["coherence"] = df["text"].apply(lambda x: check_coherence(x))
    df = get_distribution_f(df, "length_d")
    df = get_similarity_f(df)
    df = get_mean_vector_f(df, 400, model2, "_mod2")
    df = get_word2vec_features(df)
    df = get_emotional_scores(df)
    df = get_sentiment_features(df)
    clf = ParserSarcasmDetector(14)
    df["iws"] = df["text"].apply(lambda x: get_iws(x, clf))
    df = get_punctuation_and_sp_simbols_feature(df)
    df = df.drop("coherence", axis=1)
    return df

In [216]:
data_f = create_features(data)

In [217]:
def evaluation(clf, trains, tests, ans_trains, ans_tests, proba):
    f_scores = []
    recall_scores = []
    precision_scores = []
    accuracy_scores = []
    auc_scores = []
    predictions = []
    for train, test, y_train, y_test in tqdm(zip(trains, tests, y_trains, y_tests)):
        clf.fit(train, y_train)
        y_pred = clf.predict(test)
        if proba:
            y_proba = clf.predict_proba(test)
            #y_pred = get_pred(y_proba, thr)
            auc_scores.append(roc_auc_score(y_test, y_proba[:,1]))
        predictions.append(y_pred)
        f_scores.append(f1_score(y_test, y_pred))
        recall_scores.append(recall_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred))
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        
    return f_scores, recall_scores, precision_scores, accuracy_scores, auc_scores, predictions

In [218]:
def create_tt_sets(trains, test, columns_f):
    new_trains = []
    new_tests = []
    for train, test in tqdm(zip(trains, test)):
        scaler = StandardScaler()
        s_train = scaler.fit_transform(train[columns_f].astype(float))
        s_test = scaler.transform(test[columns_f].astype(float))
        new_trains.append(s_train)
        new_tests.append(s_test)
    return new_trains, new_tests

In [219]:
def get_trains_tests(data_f):
    skf = KFold(len(data_f), n_folds=10, shuffle=False, random_state=None)
    trains = []
    tests = []
    y_trains = []
    y_tests = []
    for train, test in skf:
        trains.append(data_f.iloc[train]) 
        tests.append(data_f.iloc[test])
        y_trains.append(data_f.sarc.iloc[train])
        y_tests.append(data_f.sarc.iloc[test])
    return trains, tests, y_trains, y_tests

In [220]:
trains, tests, y_trains, y_tests = get_trains_tests(data_f)

In [223]:
clf = svm.SVC(C=2.1, probability=True)
svm_features = data_f.columns.difference(set(["text",'tagged_words','words', "sarc", "sim_d", "v"]))
svm_trains, svm_tests = create_tt_sets(trains, tests, svm_features)
svm_f1, svm_recall, svm_pr, svm_acc, svm_auc, svm_predict = evaluation(clf, svm_trains, svm_tests,
                                                                       y_trains, y_tests, True)

In [224]:
print "svm avg f1:", np.mean(svm_f1)
print "svm avg recall:", np.mean(svm_recall)
print "svm avg precision:", np.mean(svm_pr)
print "svm avg accuracy:", np.mean(svm_acc)
print "svm avg auc:", np.mean(svm_auc)

svm avg f1: 0.783096988418
svm avg recall: 0.805587395072
svm avg precision: 0.762138818575
svm avg accuracy: 0.7771219171
svm avg auc: 0.855148606635


In [227]:
def load_data_sets():
    trains = []
    tests = []
    y_trains = []
    y_tests = []
    for i in tqdm(range(10)):
        train = pd.read_csv("data_train_kf_"+str(i)+".csv", encoding="utf-8")
        test = pd.read_csv("data_test_kf_"+str(i)+".csv", encoding="utf-8")
        x_train = create_features(train)
        trains.append(x_train)
        y_train = x_train.sarc
        y_trains.append(y_train)
        x_test = create_features(test)
        tests.append(x_test)
        y_test = x_test.sarc
        y_tests.append(y_test)
    return trains, tests, y_trains, y_tests 

In [228]:
trains, tests, y_trains, y_tests = load_data_sets()

In [229]:
clf = svm.SVC(C=2.1, probability=True)
svm_features = data_f.columns.difference(set(["text",'tagged_words','words', "sarc", "sim_d", "v"]))
svm_trains, svm_tests = create_tt_sets(trains, tests, svm_features)
svm_f1, svm_recall, svm_pr, svm_acc, svm_auc, svm_predict = evaluation(clf, svm_trains, svm_tests,
                                                                       y_trains, y_tests, True)

In [235]:
def store_answers(alg_name, prd):
    i = 0
    for y_pr in prd:
        pd.Series(list(y_pr)).to_csv(alg_name +"_y_prd_kf_"+str(i)+".csv")
        i += 1

In [236]:
store_answers("new", svm_predict)

In [237]:
print "svm avg f1:", np.mean(svm_f1)
print "svm avg recall:", np.mean(svm_recall)
print "svm avg precision:", np.mean(svm_pr)
print "svm avg accuracy:", np.mean(svm_acc)
print "svm avg auc:", np.mean(svm_auc)

svm avg f1: 0.782383742664
svm avg recall: 0.803472599496
svm avg precision: 0.762758242897
svm avg accuracy: 0.776680090885
svm avg auc: 0.855627349172
